In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [ ]:
#Filter BY CO2 emissions (metric tons per capita)
specific_string = 'CO2 emissions (metric tons per capita)'
filtered_rows = Train[Train['Indicator'] == specific_string]
filtered_rows

In [ ]:
#Drop Indicator Column
filtered_rows.drop(columns=['Indicator'], inplace=True)
filtered_rows

In [ ]:
#Drop Country Code Column
filtered_rows.drop(columns=['Country Code'], inplace=True)
filtered_rows

In [ ]:
#Covert Year Column in rows
melted_df = filtered_rows.melt(id_vars='Country Name', var_name='Year', value_name='CO2_Emissions')
melted_df

In [ ]:
#Sort Row in accending order by Year and Country Name
sorted_df = melted_df.groupby('Country Name').apply(lambda x: x.sort_values('Year')).reset_index(drop=True)
sorted_df.head(20)

In [ ]:
#All country name into unique_values
unique_values = melted_df['Country Name'].unique()
unique_values.shape

In [ ]:
# Replace ".." with a new value
new_value = 0.00
sorted_df = sorted_df.replace('..', new_value)

In [ ]:
sorted_df.head(15)

In [ ]:
#Remove unnecessary character.
import re
def remove_brackets(text):
    return re.sub(r"\[.*?\]", "", text)

for index, row in sorted_df.iterrows():
    sorted_df.at[index, 'Year'] = remove_brackets(row['Year'])
print(sorted_df)

In [ ]:
#Converting a Year to Datetime Format
sorted_df['Year'] = sorted_df['Year'].str.strip()
sorted_df['Year'] = pd.to_datetime(sorted_df['Year'], format='%Y')

In [ ]:
afg=sorted_df.head(16)

In [ ]:
afg

In [ ]:
afg=afg[['CO2_Emissions']]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load your yearly time series data
# Assuming your data is stored in a pandas DataFrame with a single column named 'value'

for j in unique_values:
    afg = sorted_df[sorted_df['Country Name'] == j]
    
    
    print(j)
    country_data=afg['CO2_Emissions'].astype(float)
    country_data_array = country_data.values
    country_data_array = country_data_array.reshape(-1, 1)
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(country_data_array)

    # Prepare the data for LSTM
    def create_dataset(data, time_steps):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)

    # Choose the number of time steps
    time_steps = 15 # Using data from the past 16 years to predict the next 10 years

    # Create the dataset
    X, y = create_dataset(scaled_data, time_steps)

    # Reshape input data for LSTM
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam',loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=50, batch_size=32)

    # Forecasting for the next 10 years
    forecast = []

     # Use the last `time_steps` data points from the original data to start forecasting
    current_batch = scaled_data[-time_steps:].reshape((1, time_steps, 1))

    for i in range(16):
       # Predict the next year
       next_year = model.predict(current_batch)[0, 0]
       # Append the predicted value to the forecast
       forecast.append(next_year)
       # Update the current batch with the predicted value
       current_batch = np.append(current_batch[:, 1:, :], np.array([[[next_year]]]), axis=1)
       # Reshape the predicted value to match the shape of the current batch
       next_year = np.array(next_year).reshape(1, 1, 1)

       # Inverse transform the forecast to get actual values
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    row_index = sample_submission[sample_submission.eq(j).any(axis=1)].index[0]


    new_values = [j, forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast[15][0]]
    sample_submission.loc[row_index] = new_values
       


In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)